# Spotify Streaming History Dashboard
## Part 1: Acquiring Data and Track URIs
Summary of process:
- Create dataframe from streaming history json files
- Clean dataframe 
- Populate dataframe with track URIs from library json file
- Populate remaing track URIs from Spotify Web API
***
### Install and Import Required Libraries

In [1]:
!pip install spotipy
!pip install requests
!pip install pandas
!pip install tqdm

In [19]:
import pandas as pd
import numpy as np
import requests
import json
import random
from tqdm.notebook import tqdm_notebook
import time
import spotipy
from spotipy.oauth2 import SpotifyOAuth

### Compile Database of Streaming History

In [3]:
# Put each streaming history json file into pd dataframe
df_file0 = pd.read_json('StreamingHistory0.json')
df_file1 = pd.read_json('StreamingHistory1.json')
df_file2 = pd.read_json('StreamingHistory2.json')
df_file3 = pd.read_json('StreamingHistory3.json')
df_file4 = pd.read_json('StreamingHistory4.json')

# Combine streaming history into one dataframe
df_streams = pd.concat([df_file0, df_file1, df_file2, df_file3, df_file4]).reset_index(drop=True)

# Put library json file into pd dataframe
with open('YourLibrary.json') as json_data:
    library = json.load(json_data)
df_library = pd.DataFrame(library['tracks'])

In [4]:
df_streams.tail()

,endTime,artistName,trackName,msPlayed
42798,2022-05-21 19:01,Wafia,Flowers & Superpowers,15166
42799,2022-05-21 19:01,Hippo Campus,Passenger,4596
42800,2022-05-21 19:01,HONNE,NOW I'M ALONE (feat. Sofía Valdés),56887
42801,2022-05-21 19:06,Petit Biscuit,Waterfall,18925
42802,2022-05-21 19:06,BANKS,Under The Table,249459


In [5]:
df_library.head()

,artist,album,track,uri
0,Charlotte Cardin,Phoenix Deluxe,Sex to Me,spotify:track:6HKtRka0CohhwAVKlTAAJH
1,cehryl,Delusions,Fractals,spotify:track:0rb8uwjyGc37ZPuvLBNGrj
2,Mac Miller,Circles,Woods,spotify:track:3Qa944OTMZkg8DHjET8JQv
3,BANKS,III,Godless,spotify:track:1eTL8yocTmire9gpOn2H2g
4,Billie Eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",goodbye,spotify:track:3LgWsmilsrWXiPYQFRD0T7


### Clean Dataframe and Prep for API

In [6]:
# Drop entries with unknown artist or unknown track
df_streams.drop(df_streams[df_streams.artistName=='Unknown Artist'].index, inplace=True)
df_streams.drop(df_streams[df_streams.trackName=='Unknown Track'].index, inplace=True)
df_streams.reset_index(drop=True, inplace=True)
df_streams.shape

# Remove songs with 0ms listening time
df_streams.drop(df_streams[df_streams.msPlayed==0].index, inplace=True)
df_streams.reset_index(inplace=True, drop=True)

# Remove extra days (we want exactly one year)
df_streams.drop(df_streams[df_streams.endTime<'2021-05-21'].index, inplace=True)
df_streams.drop(df_streams[df_streams.endTime>'2022-05-21'].index, inplace=True)
df_streams.reset_index(inplace=True, drop=True)

# Add URI column 
df_streams['uri'] = np.nan

In [7]:
df_streams.tail()

,endTime,artistName,trackName,msPlayed,uri
41392,2022-05-20 23:10,Your Smith,Ooh Wee,110189,NaN
41393,2022-05-20 23:10,Mac Miller,Ladders,17904,NaN
41394,2022-05-20 23:15,Jessie Reyez,"Figures, a Reprise",4110,NaN
41395,2022-05-20 23:15,Daniel Caesar,OPEN UP,266226,NaN
41396,2022-05-20 23:30,TENDER,Can't Show My Face,87796,NaN


In [8]:
# Remove "spotify:track:" from URI in the library dataframe
df_library['uri'] = df_library["uri"].str.split(":", expand = True)[2]
df_library.head()

,artist,album,track,uri
0,Charlotte Cardin,Phoenix Deluxe,Sex to Me,6HKtRka0CohhwAVKlTAAJH
1,cehryl,Delusions,Fractals,0rb8uwjyGc37ZPuvLBNGrj
2,Mac Miller,Circles,Woods,3Qa944OTMZkg8DHjET8JQv
3,BANKS,III,Godless,1eTL8yocTmire9gpOn2H2g
4,Billie Eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",goodbye,3LgWsmilsrWXiPYQFRD0T7


In [9]:
# Populate URI column in streams df with URIs from library column
for s_i, s_row in tqdm_notebook(df_streams.iterrows()):
    for l_i, l_row in df_library.iterrows():
        if ((str(s_row['artistName']) == str(l_row['artist'])) and (str(s_row['trackName']) == str(l_row['track']))):
            df_streams['uri'][s_i] = df_library['uri'][l_i]        
            
df_streams.tail(15)

0it [00:00, ?it/s]

/var/folders/75/xf0bc_w13_l1hgbk2ljvmjf40000gn/T/ipykernel_19214/2551853950.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_streams['uri'][s_i] = df_library['uri'][l_i]
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,endTime,artistName,trackName,msPlayed,uri
41382,2022-05-20 22:21,Stefflon Don,16 Shots,224727,NaN
41383,2022-05-20 22:24,MEDUZA,Lose Control,168387,NaN
41384,2022-05-20 22:24,Zara Larsson,I Would Like,14372,NaN
41385,2022-05-20 22:28,VÉRITÉ,save up,230794,NaN
41386,2022-05-20 22:32,BANKS,Burn,205970,NaN
41387,2022-05-20 22:33,Amber Mark,Waiting - Demo,101475,NaN
41388,2022-05-20 22:51,Mac Miller,2009,308871,NaN
41389,2022-05-20 23:04,Mac Miller,2009,40265,NaN
41390,2022-05-20 23:08,Michl,Self,212957,NaN
41391,2022-05-20 23:08,Drake,Trust Issues,19937,NaN


In [10]:
# Save to CSV
df_streams.to_csv('StreamingHistory_PreURI.csv', index=False)

In [11]:
# Check number of requests required for API
df_streams['uri'].isnull().sum()

22411

### Retrieve Missing URIs from Spotify Web API

In [12]:
df=pd.read_csv('StreamingHistory_PreURI.csv')

In [24]:
# Setup access for Spotify API
client_id = 'ebd35cded9324038b57d33aa816040fe'
client_secret = '4d5c594f164e475e89cb8860506c3a3e'
redirect_uri = 'http://localhost:5000/callback'
scope = 'user-read-recently-played'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirect_uri,
                                               scope=scope))

In [25]:
# Split df in four due to avoid timeout
df1 = df.iloc[0:11000]
df2 = df.iloc[11000:22000]
df3 = df.iloc[22000:33000]
df4 = df.iloc[33000:]


# Request URIs for first quarter
trackURI = []

for index, row in tqdm_notebook(df1.iterrows()):
    artist = row['artistName']
    track = row['trackName']
    uri = row['uri']
    if index % 200 == 0:
        time.sleep(random.uniform(3, 6))
    if pd.isnull(uri):
        query = sp.search(q='artist:' + artist + ' track:' + track, type='track')
        try:
            track_id = query['tracks']['items'][0]['id']
            trackURI.append(track_id)
        except:
            trackURI.append(np.nan)
    else:
        trackURI.append(uri)

0it [00:00, ?it/s]

In [26]:
# Request URIs for second quarter
for index, row in tqdm_notebook(df2.iterrows()):
    artist = row['artistName']
    track = row['trackName']
    uri = row['uri']
    if index % 200 == 0:
        time.sleep(random.uniform(3, 6))
    if pd.isnull(uri):
        query = sp.search(q='artist:' + artist + ' track:' + track, type='track')
        try:
            track_id = query['tracks']['items'][0]['id']
            trackURI.append(track_id)
        except:
            trackURI.append(np.nan)
    else:
        trackURI.append(uri)

0it [00:00, ?it/s]

In [27]:
# Request URIs for third quarter
for index, row in tqdm_notebook(df3.iterrows()):
    artist = row['artistName']
    track = row['trackName']
    uri = row['uri']
    if index % 200 == 0:
        time.sleep(random.uniform(3, 6))
    if pd.isnull(uri):
        query = sp.search(q='artist:' + artist + ' track:' + track, type='track')
        try:
            track_id = query['tracks']['items'][0]['id']
            trackURI.append(track_id)
        except:
            trackURI.append(np.nan)
    else:
        trackURI.append(uri)

0it [00:00, ?it/s]

In [28]:
# Request URIs for fourth quarter
for index, row in tqdm_notebook(df4.iterrows()):
    artist = row['artistName']
    track = row['trackName']
    uri = row['uri']
    if index % 200 == 0:
        time.sleep(random.uniform(3, 6))
    if pd.isnull(uri):
        query = sp.search(q='artist:' + artist + ' track:' + track, type='track')
        try:
            track_id = query['tracks']['items'][0]['id']
            trackURI.append(track_id)
        except:
            trackURI.append(np.nan)
    else:
        trackURI.append(uri)

0it [00:00, ?it/s]

In [29]:
# Update URI column
df['uri'] = trackURI
df.tail(15)

,endTime,artistName,trackName,msPlayed,uri
41382,2022-05-20 22:21,Stefflon Don,16 Shots,224727,2169IKAivSUUFGcedGASXc
41383,2022-05-20 22:24,MEDUZA,Lose Control,168387,7CHi4DtfK4heMlQaudCuHK
41384,2022-05-20 22:24,Zara Larsson,I Would Like,14372,4Q4jmPHwu0wrJvqrld0FQ6
41385,2022-05-20 22:28,VÉRITÉ,save up,230794,6Ce5PkT2ksqbjrRmpf30UY
41386,2022-05-20 22:32,BANKS,Burn,205970,7M2D3kwze7HQp99LGrbJlJ
41387,2022-05-20 22:33,Amber Mark,Waiting - Demo,101475,6sqjaO9q52sNKJ8S4x53DD
41388,2022-05-20 22:51,Mac Miller,2009,308871,6dFn6my1sHK2bcf23GlHwM
41389,2022-05-20 23:04,Mac Miller,2009,40265,6dFn6my1sHK2bcf23GlHwM
41390,2022-05-20 23:08,Michl,Self,212957,15dbSUtZxG2m8zk6DdZnQe
41391,2022-05-20 23:08,Drake,Trust Issues,19937,4Wjhj0WjkyECccfHVIgaTq


In [30]:
# Remove entries with no URI
df.dropna(subset=['uri'], inplace=True)
df.reset_index(inplace=True, drop=True)
df = df.astype({"uri": str})

### Save to CSV

In [31]:
df.to_csv('StreamingHistory_PostURI.csv', index=False)